In [20]:
import pandas as pd
import numpy as np
import altair as alt

In this notebook we will first attempt to fit the models to the entire dataset, ignoring positions. This may work, however, we may see trends in our more specific models where certain stats are more impactful for All-NBA statues (i.e. defensive stats may be more important for Centers as opposed to guards)

# Data

In [21]:
nba_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_szn_train.csv')
nba_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_szn_test.csv')

First we can examine the proportions of All-NBA in our dataset. We created the test and training set so that 9 random seasons were in the test set, and the rest were in training. This means our proportions should be similar for the classes in each set.

For the training set we have:

In [22]:
#Proportions of all_nba_c_year
nba_train['all_nba_c_year'].value_counts(normalize=True)

0    0.967329
1    0.032671
Name: all_nba_c_year, dtype: float64

For the testing stat we have:

In [23]:
nba_test['all_nba_c_year'].value_counts(normalize=True)

0    0.96696
1    0.03304
Name: all_nba_c_year, dtype: float64

Both sets have similar proportions as expected, but clearly we have an incredibly unbalanced dataset. To account for this, we may tune the `class-weights` parameter which will place more weights on the All-NBA class in order to account for this imbalance

### Data Filtering

We will filter our data based on variables like minutes played (`MP`), games played (`G`), since we know that these awards go to the best players in the NBA, and the best players tend to play a lot. In the new 2023 CBA (collective bargaining agreement), there is a minimum game requirement (65 games) that must be met in order to win All-NBA. However, since this rule was not in place for prior awards (where this data comes from), we can instead filter so that we only consider players who have played more games than the players with the least minutes played and games played that still won All-NBA.

In [24]:
min_minutes = nba_train[(nba_train['all_nba_c_year']==1)].MP.min()
min_G = nba_train[(nba_train['all_nba_c_year']==1)].G.min()
nba_filt_train = nba_train[(nba_train['MP']>=min_minutes) & (nba_train['G']>=min_G)]
nba_filt_test = nba_test[(nba_test['MP']>=min_minutes) & (nba_test['G']>=min_G)]

y_train = nba_filt_train['all_nba_c_year']

y_test = nba_filt_test['all_nba_c_year']

First we may fit a simpler model. Which players will make all-nba, versus which players won't given their current season stats? In this case we will ignore teams and instead only focus on the binary indicator. We may extract predicted teams by ordering the probabilities and constructing the teams in that way.

# Logistic Model

First we can describe the model of interest under a statistical framework. Denote the following quantities:
1) y: an n x 1 vector, containing the binary variable of interest
2) X: and n x (p+1) matrix, consisting of the feature variables and intercept
3) $\beta$: A (p+1) x 1 vector of coefficients.

We will also use the following functions:

1) $logit(p) = log(\frac{p}{1-p})$; this is often denoted as the log-odds
2) $expit(x) = \frac{1}{1+exp(-x)}$; this is the inverse function of logit, i.e. $expit(x) = logit^{-1}(x)$

The model we will be utilizing is:
$$
y_i \sim Bernoulli(p_i = expit(x_i^{T}\beta))
$$

Where $x_i$ is the i'th row of the feature matrix X.

If we assume independence (clearly broken here since player performance is clearly correlated across different seasons, but we will disregard this for now), then we have a likelihood function of the form:

$$
L(\beta) = \prod_{i=1}^n (expit(x_i^{T}\beta))^{y_i}\times (1-expit(x_i^{T}\beta))^{1-y_i}
$$

Leading to a log-likelihood function (our unregularized negative objective function) of:

$$
\ell(\beta) = \sum_{i=1}^n y_i(log(expit(x_i^{T}\beta))) + (1-y_i)log(1-expit(x_i^{T}\beta)) \\ = \sum_{i=1}^n y_i(x_i^T\beta)-log(1+exp(x_i^T\beta))
$$



Thus we will be finding:

$$
\underset{\beta}{min}\sum_{i=1}^n -y_i(x_i^T\beta)+log(1+exp(x_i^T\beta)) + r(\beta)
$$

where $r(\beta)$ is a regularization term

For the L1 regularizer SKlearn specifically will be minimizing:
$$
\underset{\beta}{min} \ C\sum_{i=1}^n -y_i(x_i^T\beta)+log(1+exp(x_i^T\beta)) + \sum_{i=0}^p|\beta_i|
$$

Where the regularizing constant is given as C>0. This is a hyperparameter we must tune

One issue with this model, however, is that it does not know that only 15 (10 for year $\leq$ 1988) are selected for All-NBA. So for our predictions, we will take the top 6 players in the G category, top 6 in F, and top 3 in C (4, 4, 2 for year $\leq$ 1988) ranking them by their outputted probabilities from the model. For cross validation (for tuning C), we will use the F1 score of these modified predictions

### Fitting Logistic Model to NBA Data

We will select the best C based off of k-fold cross validation, with k=5. We will also tune the class-weights as well.

In [25]:
#Create custom scoring function for CV
from sklearn.metrics import f1_score
def predicted_all_nba(nba_test_df, model):
    test_df = nba_test_df.copy()
    test_df.loc[:,'prob_all_nba'] = model.predict_proba(test_df)[:,1]
    years = test_df['year'].unique()
    test_df['pred_all_nba'] = 0
    test_df['4th Team'] = 0
    for year in years:

        if year <= 1988:
            G_all_nba = test_df[(test_df['year']==year) & (test_df['Position']=='G')].sort_values(by='prob_all_nba', ascending=False).head(6)['Player'].tolist()

            F_all_nba = test_df[(test_df['year']==year) & (test_df['Position']=='F')].sort_values(by='prob_all_nba', ascending=False).head(6)['Player'].tolist()
            
            C_all_nba = test_df[(test_df['year']==year) & (test_df['Position']=='C')].sort_values(by='prob_all_nba', ascending=False).head(3)['Player'].tolist()
            
            all_nba_players = G_all_nba[0:4] + F_all_nba[0:4] + C_all_nba[0:2]
            all_nba_players_4 = G_all_nba[4:6] + F_all_nba[4:6] + C_all_nba[2:3]
            test_df.loc[(test_df['year']==year) & (test_df['Player'].isin(all_nba_players)), 'pred_all_nba'] = 1
            test_df.loc[(test_df['year']==year) & (test_df['Player'].isin(all_nba_players_4)), '4th Team'] = 1
        else:
            G_all_nba = test_df[(test_df['year']==year) & (test_df['Position']=='G')].sort_values(by='prob_all_nba', ascending=False).head(8)['Player'].tolist()

            F_all_nba = test_df[(test_df['year']==year) & (test_df['Position']=='F')].sort_values(by='prob_all_nba', ascending=False).head(8)['Player'].tolist()
            
            C_all_nba = test_df[(test_df['year']==year) & (test_df['Position']=='C')].sort_values(by='prob_all_nba', ascending=False).head(4)['Player'].tolist()
            
            all_nba_players = G_all_nba[0:6] + F_all_nba[0:6] + C_all_nba[0:3]
            all_nba_players_4 = G_all_nba[6:8] + F_all_nba[6:8] + C_all_nba[3:4]

            test_df.loc[(test_df['year']==year) & (test_df['Player'].isin(all_nba_players)), 'pred_all_nba'] = 1
            test_df.loc[(test_df['year']==year) & (test_df['Player'].isin(all_nba_players_4)), '4th Team'] = 1

    return test_df[['Player', 'year','Position', 'all_nba_tm','prob_all_nba','pred_all_nba','all_nba_c_year','4th Team']]


from sklearn.metrics import f1_score

def all_nba_f1(model, X, y):
    y_pred = predicted_all_nba(X, model)['pred_all_nba']
    return f1_score(y, y_pred)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#removing linearly dependent features
#FG = 2P+3P
#FGA = 2PA+3PA
#TRB = ORB+DRB
#WS = OWS+DWS
#BPM = OBPM+DBPM
num_features = ['Age','G', 
                'GS', 'MP',
                'FG', 'FGA', 
                'FG%', 
                '3P', '3PA', '3P%',
                '2P', '2PA', 
                '2P%', 'eFG%', 
                'FT', 'FTA', 'FT%', 
                'ORB', 
                'DRB', 
                'TRB',
                'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 
                'TS%', '3PAr', 'FTr',
                'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 
                'OWS',
                'DWS',
                'WS', 
                'WS/48', 
                'OBPM', 
                'DBPM', 
                'BPM', 
                'VORP', 'W',
                'num_all_nba',
                'seed'
                ]

#cat_features = ['Tm']

#Now I will create a pipeline where I extract my_features, and apply OHE to cat_features
ct = ColumnTransformer(
    [("select", "passthrough", num_features),
     #("ohe", OneHotEncoder(handle_unknown="ignore"), cat_features)
     ],
     remainder="drop"
)



clf1 = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0
                                      ))
])

#will create parameter grid for gridsearch for C and class_weight
#Setting the range for class weights
log_weights = np.linspace(0,1,30)

c_grid = [0.001, 0.01, 0.1, .2, .3, .4, .5, .7, .8, 1, 3, 5 ,10, 20, 50, 100, 1000]

log_param_grid = {
    'classifier__C': c_grid,
    'classifier__class_weight': [{0: x, 1: 1.0-x} for x in log_weights]
}

#do gridsearch using all_nba_f1 as scoring metric
log_model = GridSearchCV(clf1, log_param_grid, cv=5,              
                           scoring=all_nba_f1,
                           n_jobs=-1)
log_model.fit(nba_filt_train, y_train)

In [ ]:
log_model.best_params_

### Logistic Results

We now have the following metrics for our model.

In [ ]:
log_preds = predicted_all_nba(nba_filt_test, log_model)['pred_all_nba']
log_all_preds = nba_filt_test.copy()
log_all_preds['pred_all_nba'] = log_preds
log_all_preds.to_csv("test_preds/log_preds_all.csv")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, log_preds))

We see that our precision and recall are not too bad for this model. Precision is the percentage of predicted positives that were true positives (how good are your guesses), and recall is the the percentage of predicted positives from the truth positives (how many of the positives did you correctly identify).

Looking at the confusion matrix for these results we see that we had around 20 false postives, and false negatives. These are not ideal, but considering the size of our dataset, may still work for our purposes.

In [ ]:
pd.crosstab(y_test,  log_preds, rownames=["Actual"], colnames=["Predicted"])

Looking at the total coefs and the nonzero coefs also see that our L1 penalty greatly reduced the numbe of features in our dataset

In [ ]:
#Now we will extract the feature names from the pipeline
feature_names = log_model.best_estimator_.named_steps['col_transform'].get_feature_names_out()
coef_df = pd.DataFrame({'coef':log_model.best_estimator_['classifier'].coef_[0]
                        ,'var':feature_names})

In [ ]:
coef_df_nz = coef_df[coef_df['coef']!=0]

We see from our coefficient plot that FGA and wins seems to have the highest impact on the log-odds of winning All-NBA, while FT% has the biggest negative impact. This bears further study, but for the purposes of this project (prediction), we do not necessarily care about how the model is weighing each of the features

In [ ]:
#Now we will make a bar chart of these coefficients
alt.Chart(coef_df_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

We see for this dataset we have the following players who were predicted All-NBA but did not win it. (False Positives)

In [ ]:
nba_filt_test[(log_preds==1) & (y_test!=1)][['Player',"year"]].sort_values(by='year', ascending=False)

For our False Negatives we have:

In [ ]:
nba_filt_test[(log_preds!=1) & (y_test==1)][['Player',"year"]].sort_values(by='year', ascending=False)

We can also examine what the 2023 predicted All_NBA team looks like versus the actual team.

In [ ]:
def pred_all_nba_df(nba_test, year, model):
    if year<=1988:
        return 'year must be above 1988'
    pred_df = predicted_all_nba(nba_test, model)
    pred_df = pred_df[pred_df['year']==year]
    pred_4 = pred_df[pred_df['4th Team']==1]
    pred_df = pred_df[pred_df['pred_all_nba']==1]
    pred_df.sort_values(by=['Position','prob_all_nba'], ascending=False, inplace=True)
    pred_4.sort_values(by='Position', ascending=False, inplace=True)
    pred_team1 = pred_df.iloc[[0,1,6,7,12],:]['Player'].tolist()
    pred_team2 = pred_df.iloc[[2,3,8,9,13],:]['Player'].tolist()
    pred_team3 = pred_df.iloc[[4,5,10,11,14],:]['Player'].tolist()
    pred_team4 = pred_4['Player'].tolist()
    all_nba_df = pd.DataFrame({'Predicted Team 1':pred_team1, 'Predicted Team 2':pred_team2, 'Predicted Team 3':pred_team3, 'Predicted Team 4':pred_team4})
    all_nba_df.index = ['Guard', 'Guard', 'Forward', 'Forward', 'Center']
    return all_nba_df.transpose()

def true_all_nba(nba_test,year):
    teams = ['1st','2nd','3rd']
    if year<=1988:
        return 'year must be above 1988'
    year_all_nba = nba_test[(nba_test['year']==year) & (nba_test['all_nba_tm'].isin(teams))].sort_values(by=['all_nba_tm','Position'], ascending=True)
    team1 = year_all_nba.iloc[np.arange(0,5),:]['Player'].tolist()
    team2 = year_all_nba.iloc[np.arange(5,10),:]['Player'].tolist()
    team3 = year_all_nba.iloc[np.arange(10,15),:]['Player'].tolist()
    all_nba_df = pd.DataFrame({'True Team 1':team1, 'True Team 2':team2, 'True Team 3':team3})
    all_nba_df.index = ['Center', 'Forward', 'Forward', 'Guard', 'Guard']
   
    return  all_nba_df.iloc[::-1].transpose()


In [ ]:
true_all_nba(nba_filt_test, 2023)

In [ ]:
pred_all_nba_df(nba_filt_test, 2023, log_model)

# Random Forest Model

A random forest is considered an "Ensemble" Model. These types of models take in a large number of different models that are performing slightly better than chance, and then combine them into a stronger model. For a random forest, this means that `n-estimator` total decision trees are created, then their results are combined (by  picking the result the majority of trees agree with). Each tree is trained on random subsets of the data, with random subsets of the features.

This also begs the question, why do these models work? We can consider a simple example, where we can imagine that we have a `n` total models that predict the correct results, say 51% of the time. If we assume each model is independent in their predictions, then we can utilize a bernoulli model and apply the law of large numbers. In simpler terms, we can imagine every model is like a coin flip, where the true result is a heads (with 51% chance), and the incorrect result is tails (49%). If we consider an ensemble of 10 models, then we will simply consider how many heads we have vs tails. If we have more heads, we pick that result, and more tails we pick the other result. Clearly, this will lead to quite a bit of variance in our results, and we will often pick the wrong result. But imagine we are flipping 1000 coins, or even 100000 coins. We see that we will have, with probability approaching 1, that as our number of models grow, the number of heads will be greater than the number of tails. In our case, the independence assumption is violated since we utilize the same data, however, we can still yield good results even without this assumption. 

### Fitting Random Forest Model to NBA Data

To fit this model we will consider a grid of hyper-parameters. Since this grid is quite large (7200), we will utilize a randomized search where we consider 500 random subsets of the hyper-parameters.

We can also consider what these hyper-parameters mean.
1) n_estimators: How many decision trees will be created in total
2) max_depth: The longest path between the root node and the leaf. This is essentially controlling how many splits are allowed.
3) min_samples_lead: Minimum number of samples to required for a leaf node. This determines how specific a leaf can be.
4) min_samples_split: Minimum number of samples a node must have for it to be split
5) max_features: Maximum number features randomly selected for each tree.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Now we will fit a random forest model to the data. We will fit this data into a pipeline to scale the data and then fit the model.

num_features = ['Age','G', 
                'GS', 'MP',
                'FG', 'FGA', 
                'FG%', 
                '3P', '3PA', '3P%',
                '2P', '2PA', 
                '2P%', 'eFG%', 
                'FT', 'FTA', 'FT%', 
                'ORB', 
                'DRB', 
                'TRB',
                'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 
                'TS%', '3PAr', 'FTr',
                'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 
                'OWS',
                'DWS',
                'WS', 
                'WS/48', 
                'OBPM', 
                'DBPM', 
                'BPM', 
                'VORP', 'W',
                'num_all_nba',
                'seed'
                ]

#cat_features = ['Tm']

#Now I will create a pipeline where I extract my_features, and apply OHE to cat_features
ct2 = ColumnTransformer(
    [("select", "passthrough", num_features),
     #("ohe", OneHotEncoder(handle_unknown="ignore"), cat_features)
     ],
     remainder="drop"
)

clf2 = Pipeline([('col_transform',ct2),
                ('classifier',RandomForestClassifier(random_state=0))
                ])

# rf_model = Pipeline([('col_transform',ct),
#                 ('classifier',RandomForestClassifier(random_state=0,
#                                                      n_estimators=200,
#                                                      min_samples_split=2,
#                                                      min_samples_leaf=5,
#                                                      max_features=.2,
#                                                      max_depth=20,
#                                                      class_weight='balanced_subsample'))
#                 ])
# rf_model.fit(nba_filt_train,y_train)


In [ ]:
#Now we can fit our Random Forest model to the data.

from sklearn.model_selection import RandomizedSearchCV
rf_param_grid = {
   'classifier__n_estimators': [100, 200, 300, 400, 500],
   'classifier__max_depth': [5, 10, 15, 20, 25],
   'classifier__min_samples_split': [2, 5, 10, 15, 20, 25, 30, 40],
   'classifier__min_samples_leaf': [5, 10, 15, 20],
   'classifier__max_features': [ 'sqrt', 'log2', 0.2, .4],
   'classifier__class_weight': ['balanced', 'balanced_subsample', None]
}


rf_model = RandomizedSearchCV(estimator=clf2, param_distributions=rf_param_grid, cv= 5,
                            random_state=0, n_iter = 3000,
                            scoring=all_nba_f1,
                            n_jobs=-1)
rf_model.fit(nba_filt_train, y_train)


In [ ]:
rf_model.best_params_

### Results

Now we may look at how our model did on our test set. 

In [ ]:
rf_preds = predicted_all_nba(nba_filt_test, rf_model)['pred_all_nba']

We have for our precision, recall, and f1-score:

In [ ]:
print(classification_report(y_test, rf_preds, target_names=['Not All-NBA', 'All-NBA']))

Looking at our confusion matrix we have the following:

In [ ]:
pd.crosstab(y_test, rf_preds, rownames=['Actual'], colnames=['Predicted'], margins=True)

We can view our false positives as:

In [ ]:
#False Positives
nba_filt_test[(rf_preds==1) & (y_test==0)][['Player','year']].sort_values(by='year', ascending=False)

For our False negatives we have:

In [ ]:
#False Negatives
nba_filt_test[(rf_preds==0) & (y_test==1)][['Player','year']].sort_values(by='year', ascending=False)

In [ ]:
feature_names = rf_model.best_estimator_.named_steps['col_transform'].get_feature_names_out()
coef_df = pd.DataFrame({'coefs':rf_model.best_estimator_['classifier'].feature_importances_,
                          'features':feature_names})
coef_df_nz = coef_df[coef_df['coefs']!=0]
coef_df_nz.sort_values(by='coefs', ascending=False, inplace=True)

We can also view which coefficients made the most impact on the model. We see similar to our logistic regression, VORP was one of the strongest features in prediciting All-NBA status. 

In [ ]:
alt.Chart(coef_df_nz[0:40]).mark_bar().encode(
    y='coefs',
    x=alt.Y('features', sort='-y'))

We can also examine what the 2023 predicted All_NBA team looks like versus the actual team.

In [ ]:
true_all_nba(nba_filt_test, 2023)

In [ ]:
pred_all_nba_df(nba_filt_test, 2023, rf_model)

# XGBoost

XGboost is a more recent classification algorithm that also makes use of ensemble learners. In this algorithm, we similarly create decision trees, but rather than each tree being independent, each successive tree tries to improve on its predecessor. This is done by trying to fit to residual values, and find any underlying patterns that may be there. The algorithm is stopped when the residuals are sufficiently random and no more patterns can be found. Similar to Random Forests, we have quite a bit of hyper parameters to tune, so a randomized CV search is utilized. 

In [ ]:
import xgboost as xgb

#Now we will fit a random forest model to the data. We will fit this data into a pipeline to scale the data and then fit the model.

num_features = ['Age','G', 
                'GS', 'MP',
                'FG', 'FGA', 
                'FG%', 
                '3P', '3PA', '3P%',
                '2P', '2PA', 
                '2P%', 'eFG%', 
                'FT', 'FTA', 'FT%', 
                'ORB', 
                'DRB', 
                'TRB',
                'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 
                'TS%', '3PAr', 'FTr',
                'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 
                'OWS',
                'DWS',
                'WS', 
                'WS/48', 
                'OBPM', 
                'DBPM', 
                'BPM', 
                'VORP', 'W',
                'num_all_nba',
                'seed'
                ]

#cat_features = ['Tm']

#Now I will create a pipeline where I extract my_features, and apply OHE to cat_features
ct3 = ColumnTransformer(
    [("select", "passthrough", num_features),
     #("ohe", OneHotEncoder(handle_unknown="ignore"), cat_features)
     ],
     remainder="drop"
)

scale_pos = (len(y_train) - sum(y_train))/sum(y_train)

clf3 = Pipeline([('col_transform',ct3),
                     ('classifier',xgb.XGBClassifier(random_state=0, scale_pos_weight = scale_pos))
                     ])
xgb_param_grid = {
    'classifier__n_estimators': [100, 200, 300, 400, 500],
    'classifier__max_depth': [5, 10, 15, 20, 25],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, .3],
    'classifier__min_child_weight': [1, 2, 5, 10, 15],
    'classifier__gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'classifier__colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7],
    'classifier__subsample': [0.3, 0.4, 0.5, 0.6, 0.7,1]
}

xgb_param_grid_test = {
    'classifier__n_estimators': [100]
}


xgb_model = RandomizedSearchCV(estimator=clf3, param_distributions=xgb_param_grid, cv= 5, n_iter = 3000,
                           random_state=0,
                           scoring=all_nba_f1,
                           n_jobs = -1
                           )

In [ ]:
xgb_model.fit(nba_filt_train, y_train)

In [ ]:
xgb_model.best_params_

### Results

In [ ]:
xgb_preds = predicted_all_nba(nba_filt_test, xgb_model)['pred_all_nba']

We have our metrics for this model as:

In [ ]:
print(classification_report(y_test, xgb_preds, target_names=['Not All-NBA', 'All-NBA']))

In [ ]:
pd.crosstab(y_test, xgb_preds, rownames=['Actual'], colnames=['Predicted'])

For our feature importance, we see that the advanced statistic PER is the most impactful, with VORP being the second most so.

In [ ]:
feature_names = xgb_model.best_estimator_.named_steps['col_transform'].get_feature_names_out()
coef_df = pd.DataFrame({'coefs':xgb_model.best_estimator_['classifier'].feature_importances_,
                          'features':feature_names})
coef_df_nz = coef_df[coef_df['coefs']!=0]

alt.Chart(coef_df_nz).mark_bar().encode(
    y='coefs',
    x=alt.Y('features', sort='-y'))

For our false positives we have:

In [ ]:
nba_filt_test[(xgb_preds==1) & (y_test!=1)][['Player',"year"]]

For our false negatives we have:

In [ ]:
nba_filt_test[(xgb_preds!=1) & (y_test==1)][['Player',"year"]]

We can also examine what the 2023 predicted All_NBA team looks like versus the actual team.

In [ ]:
true_all_nba(nba_filt_test, 2023)

In [ ]:
pred_all_nba_df(nba_filt_test, 2023, xgb_model)

# Ensemble Model

Now we will try to use an ensemble learner using all 3 methods utilized so far: Logistic Regression, Random Forests, and XGboost. We will use soft voting (averaging over all the predicted probabilities) to classify the players

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemb_clf = VotingClassifier(estimators=[('lr', log_model.best_estimator_), ('rf', rf_model.best_estimator_), ('xgb', xgb_model.best_estimator_)],
                              voting='soft', weights=[1,1,1])


In [ ]:
ensemb_clf.fit(nba_filt_train, y_train)

### Results

We have the following metrics for this ensemble learner:

In [ ]:
ens_preds = predicted_all_nba(nba_filt_test, ensemb_clf)['pred_all_nba']

In [ ]:
print(classification_report(y_test, ens_preds))

We see that we end up with similar precision, recall as our logistic regression, indicating we did not do that much better than the individual models

For our confusion matrix we have:

In [ ]:
pd.crosstab(y_test, ens_preds, rownames=['Actual'], colnames=['Predicted'], margins=True)

As a final metric we can also view the ROC curve. This plots the true positive rate vs the false positive rate. To measure how effective our classifier is, we consider area under the curve (AUC). A perfect classifier would have an AUC of 1.

In [ ]:
#Now we can draw the ROC curves for all 3 models and the ensemble model.
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
#Now we may plot them,
plt.figure(figsize=(10,10))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(*roc_curve(y_test, log_model.predict_proba(nba_filt_test)[:,1])[:2], label="Logistic Regression (area = %0.3f)" % auc(*roc_curve(y_test, log_model.predict_proba(nba_filt_test)[:,1])[:2]))
plt.plot(*roc_curve(y_test, rf_model.predict_proba(nba_filt_test)[:,1])[:2], label="Random Forest (area = %0.3f)" % auc(*roc_curve(y_test, rf_model.predict_proba(nba_filt_test)[:,1])[:2]))
plt.plot(*roc_curve(y_test, xgb_model.predict_proba(nba_filt_test)[:,1])[:2], label="XGBoost (area = %0.3f)" % auc(*roc_curve(y_test, xgb_model.predict_proba(nba_filt_test)[:,1])[:2]))
plt.plot(*roc_curve(y_test, ensemb_clf.predict_proba(nba_filt_test)[:,1])[:2], label="Ensemble(area = %0.3f)" % auc(*roc_curve(y_test, ensemb_clf.predict_proba(nba_filt_test)[:,1])[:2]))
plt.legend()
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')

auc_scores = [auc(*roc_curve(y_test, log_model.predict_proba(nba_filt_test)[:,1])[:2]),
                auc(*roc_curve(y_test, rf_model.predict_proba(nba_filt_test)[:,1])[:2]),
                auc(*roc_curve(y_test, xgb_model.predict_proba(nba_filt_test)[:,1])[:2]),
                auc(*roc_curve(y_test, ensemb_clf.predict_proba(nba_filt_test)[:,1])[:2])]
              
print("Logistic Regression AUC: ", auc_scores[0])
print("Random Forest AUC: ", auc_scores[1])
print("XGBoost AUC: ", auc_scores[2])
print("Ensemble AUC: ", auc_scores[3])

We can also examine what the 2023 predicted All_NBA team looks like versus the actual team.

In [ ]:
true_all_nba(nba_filt_test, 2023)

In [ ]:
pred_all_nba_df(nba_filt_test, 2023, ensemb_clf)

Now we may put all our results into a dataframe

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

#Putting precision and recall and ROC-AUC score into a dataframe for all 4 models
model_names = ['Logistic Regression', 'Random Forest', 'XGBoost', 'Ensemble']
precision = [0,0,0,0]
recall = [0,0,0,0]
roc_auc = [0,0,0,0]
f1_scores = [0,0,0,0]
accuracy = [0,0,0,0]
for i, preds in enumerate([log_preds, rf_preds, xgb_preds, ens_preds]):
    precision[i] = precision_score(y_test, preds)
    recall[i] = recall_score(y_test, preds)
    roc_auc[i] = auc_scores[i]
    f1_scores[i] = f1_score(y_test, preds)
    accuracy[i] = accuracy_score(y_test, preds)
res_df = pd.DataFrame({'Model': model_names, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F-1 Score':f1_scores ,'ROC-AUC': roc_auc})
res_df.to_csv('results.csv', index=False)

In [ ]:
res_df